In [1]:
!pip install transformers[torch] datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [30]:
import json
import pandas as pd
import numpy as np
import torch
from transformers import TrainingArguments, Trainer, pipeline
from pathlib import Path
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from tqdm.auto import tqdm

In [3]:
df = pd.read_csv("/content/drive/MyDrive/News_Summarization/news_summary.csv", encoding="latin-1")

In [4]:
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [5]:
df.shape

(4514, 6)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514 entries, 0 to 4513
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author     4514 non-null   object
 1   date       4514 non-null   object
 2   headlines  4514 non-null   object
 3   read_more  4514 non-null   object
 4   text       4514 non-null   object
 5   ctext      4396 non-null   object
dtypes: object(6)
memory usage: 211.7+ KB


In [7]:
df.columns

Index(['author', 'date', 'headlines', 'read_more', 'text', 'ctext'], dtype='object')

In [8]:
df = df.drop(columns=['author', 'date', 'headlines', 'read_more'], axis=1)

In [9]:
df.shape

(4514, 2)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514 entries, 0 to 4513
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4514 non-null   object
 1   ctext   4396 non-null   object
dtypes: object(2)
memory usage: 70.7+ KB


In [11]:
df = df.dropna()

In [12]:
df.shape

(4396, 2)

In [13]:
data = Dataset.from_pandas(df)
data

Dataset({
    features: ['text', 'ctext', '__index_level_0__'],
    num_rows: 4396
})

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [15]:
model_ckpt = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['ctext'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['text'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [17]:
dataset_pt = data.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/4396 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
trainer_args = TrainingArguments(
    output_dir='flan-t5', num_train_epochs=6, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=200,save_steps=1e6,
    gradient_accumulation_steps=16,
    push_to_hub=False,
    report_to=[],
)

In [20]:
trainer = Trainer(model=model,
                  args=trainer_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt)

In [21]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,1.802300
400,1.670900
600,1.569800
800,1.519100
1000,1.468700
1200,1.452100
1400,1.424000
1600,1.413600


TrainOutput(global_step=1644, training_loss=1.535742337396255, metrics={'train_runtime': 5257.2649, 'train_samples_per_second': 5.017, 'train_steps_per_second': 0.313, 'total_flos': 1.6536023415595008e+16, 'train_loss': 1.535742337396255, 'epoch': 5.98})

In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
model.push_to_hub('ErnestBeckham/flan-t5-base-news-summarization')

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ErnestBeckham/flan-t5-base-news-summarization/commit/2170c73b6f5f0b1d881785c54725be8e8a8f7762', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='2170c73b6f5f0b1d881785c54725be8e8a8f7762', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
tokenizer.push_to_hub('ErnestBeckham/flan-t5-base-news-summarization')

CommitInfo(commit_url='https://huggingface.co/ErnestBeckham/flan-t5-base-news-summarization/commit/30614299c4038651c5ae5973043c86ef5d9a79bd', commit_message='Upload tokenizer', commit_description='', oid='30614299c4038651c5ae5973043c86ef5d9a79bd', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = "Online tree guiding application is developed for to make aware people about how to choose suitable plant that suits for their area and guide that person until get the harvest, through this application. Mainly we will use prototyping methodology to implement this system. It includes all the required functions that helps to operate the online tree guidance smoothly. The user must install this application from google play store and must create only one account. It is not allowed to create more than one account for each user due to the security of the system. Users must agree for all the guidelines and regulations before start working with the application. After creating the account, user can enter the exact area which they are living and the date that they planned to plant trees and it automatically generate the status of the weather and suggest most suitable plants for that area. But user can add any type of plant that they like, by using ‘other’ option and the system will automatically generate the percentage of success that plant can grow that area. After processing all these details, it will generate the report for the user including all the details about the selected plant. Then guide the user until they get the harvest. It provides the measurements of all ingredients like water, soil, sun light etc. Moreover, if any user like to sell or buy their harvest, our system is best platform to them as well. The function call ‘store’ provide facility to connect sellers and buyers through this system. Therefore, this is also a good system to improve their businesses who use this application. It also provides error messages when the user enters wrong data. At the beginning, it will display a system tour to get some idea about the main functions of the system to the user. To use this application, user no need any formal knowledge."#dataset_pt["ctext"][0]

#reference = dataset_pt["text"][0]

pipe = pipeline("summarization", model='ErnestBeckham/flan-t5-base-summarization', tokenizer=tokenizer)
##
print("Dialogue:")
print(sample_text)


#print("\nReference Summary:")
#print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Online tree guiding application is developed for to make aware people about how to choose suitable plant that suits for their area and guide that person until get the harvest, through this application. Mainly we will use prototyping methodology to implement this system. It includes all the required functions that helps to operate the online tree guidance smoothly. The user must install this application from google play store and must create only one account. It is not allowed to create more than one account for each user due to the security of the system. Users must agree for all the guidelines and regulations before start working with the application. After creating the account, user can enter the exact area which they are living and the date that they planned to plant trees and it automatically generate the status of the weather and suggest most suitable plants for that area. But user can add any type of plant that they like, by using ‘other’ option and the system will auto